# Synthetic waves on the sphere, Healpix format

In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from scipy.special import sph_harm

def create_synthetic_waves(nside=256, lmax=None, dt=3600, nt=24, 
                          phase_speed_func=None, mean_wind=None):
    """
    Create synthetic propagating waves on sphere with prescribed phase speeds
    
    Parameters:
    - nside: HEALPix resolution (256 = level 8)
    - lmax: Maximum spherical harmonic degree
    - dt: Time step in seconds
    - nt: Number of time steps
    - phase_speed_func: Function c(l) returning phase speed for wavenumber l
    - mean_wind: Mean zonal wind field (eastward, m/s) as HEALPix map
    """
    if lmax is None:
        lmax = 2 * nside
    if phase_speed_func is None:
        # Default: gravity waves with varying phase speeds
        phase_speed_func = lambda l: 50 + 200/np.sqrt(l+1)  # m/s
    
    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    
    # Initialize random wave amplitudes and phases
    np.random.seed(42)  # Reproducible
    
    # Time series arrays
    h_maps = np.zeros((nt, npix))  # Height/temperature field
    w_maps = np.zeros((nt, npix))  # Vertical velocity field
    
    # Earth radius
    R = 6.37e6  # meters
    
    for t_idx in range(nt):
        time = t_idx * dt
        
        # Initialize maps for this time step
        h_t = np.zeros(npix)
        w_t = np.zeros(npix)
        
        # Add waves for each l, m combination
        for l in range(1, min(lmax+1, 50)):  # Limit for speed
            c_l = phase_speed_func(l)  # Phase speed for this wavenumber
            k = l / R  # Wavenumber (1/m)
            omega = k * c_l  # Angular frequency
            
            for m in range(-l, l+1, max(1, l//5)):  # Sample m values
                # Random amplitude and initial phase
                amp = np.random.exponential(1.0) * np.exp(-l/20)  # Decay with l
                phi0 = np.random.uniform(0, 2*np.pi)
                
                # Spherical harmonic
                #Ylm = hp.sphtfunc.sph_harm(m, l, phi, theta)
                Ylm = sph_harm(m, l, phi, theta)

                
                # Wave phase including propagation and Doppler shift
                phase = omega * time + phi0
                
                # Add Doppler shift if mean wind provided
                if mean_wind is not None:
                    # Approximate Doppler shift: k_eff = k + U*m/(R*cos(theta))
                    doppler = mean_wind * m / R
                    phase += doppler * time
                
                # Height field (cosine)
                h_t += amp * np.real(Ylm * np.exp(1j * phase))
                
                # Vertical velocity (sine, leading by π/2)
                w_t += amp * omega * np.real(Ylm * np.exp(1j * (phase + np.pi/2)))
        
        h_maps[t_idx] = h_t
        w_maps[t_idx] = w_t
    
    return h_maps, w_maps

def animate_waves(h_maps, w_maps, dt=3600, figsize=(12, 5)):
    """Animate the synthetic wave fields"""
    nt = h_maps.shape[0]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Initial plots
    im1 = None
    im2 = None
    
    def animate(frame):
        nonlocal im1, im2
        ax1.clear()
        ax2.clear()
        
        # Plot height field
        hp.mollview(h_maps[frame], hold=True, title=f'Height t={frame*dt/3600:.1f}h', 
                   cbar=False, fig=fig.number, sub=(1,2,1))
        
        # Plot velocity field  
        hp.mollview(w_maps[frame], hold=True, title=f'Velocity t={frame*dt/3600:.1f}h',
                   cbar=False, fig=fig.number, sub=(1,2,2))
        
        return []
    
    anim = FuncAnimation(fig, animate, frames=nt, interval=200, blit=False)
    plt.tight_layout()
    return anim

def simple_animate(h_maps, dt=3600):
    """Simple animation showing just height field"""
    nt = h_maps.shape[0]
    
    fig = plt.figure(figsize=(8, 6))
    
    def animate(frame):
        plt.clf()
        hp.mollview(h_maps[frame], title=f'Synthetic Waves t={frame*dt/3600:.1f}h')
    
    anim = FuncAnimation(fig, animate, frames=nt, interval=300, repeat=True)
    return anim

In [4]:
# Create synthetic data
h_maps, w_maps = create_synthetic_waves(nside=42, nt=3)

# Simple animation
#anim = simple_animate(h_maps)
#plt.show()

# To save as files:
for i, (h, w) in enumerate(zip(h_maps, w_maps)):
     hp.write_map(f'synthetic_h_{i:03d}.fits', h)
     hp.write_map(f'synthetic_w_{i:03d}.fits', w)

/glade/derecho/scratch/mapes/tmp/ipykernel_74460/3719182378.py:61: DeprecationWarning: `scipy.special.sph_harm` is deprecated as of SciPy 1.15.0 and will be removed in SciPy 1.17.0. Please use `scipy.special.sph_harm_y` instead.
  Ylm = sph_harm(m, l, phi, theta)
setting the output map dtype to [dtype('float64')]


ValueError: cannot reshape array of size 21168 into shape (20,1024)

In [ ]:
# Example usage:
if __name__ == "__main__":
    # Create synthetic data
    h_maps, w_maps = create_synthetic_waves(nside=256, nt=20)
    
    # Simple animation
    anim = simple_animate(h_maps)
    plt.show()
    

In [ ]:
    # To save as files:
    for i, (h, w) in enumerate(zip(h_maps, w_maps)):
        hp.write_map(f'synthetic_h_{i:03d}.fits', h)
        hp.write_map(f'synthetic_w_{i:03d}.fits', w)